In [ ]:
!pip install scikit-learn
#!pip install tensorflow

In [13]:
import tensorflow as tf
print (tf.__version__)

2.7.0


In [2]:
import os
import pandas as pd 
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import valohai

In [15]:
DATA_CSV_FILE = os.path.join(os.getcwd(), "Notebooks", "Iris.csv")
print (DATA_CSV_FILE)

/home/choprasu/Desktop/ml/test_valohai/Notebooks/Notebooks/Iris.csv


In [4]:
# Parameters
learning_rate = 0.001
epochs = 5
batch_size = 100

In [5]:
iris = pd.read_csv(DATA_CSV_FILE)
iris.drop("Id", axis=1, inplace = True)

In [6]:
x = iris[['SepalLengthCm','SepalWidthCm','PetalLengthCm','PetalWidthCm']].values
y_ = iris.Species.values

In [7]:
y_ = y_.reshape(-1, 1) # Convert data to a single column

# One Hot encode the class labels
encoder = OneHotEncoder(sparse=False)
y = encoder.fit_transform(y_)
#print(y)

# Split the data for training and testing
train_x, test_x, train_y, test_y = train_test_split(x, y, test_size=0.20)

In [8]:
print(train_x.shape)
print(train_y.shape)

(120, 4)
(120, 3)


In [9]:
class CustomCallback(tf.keras.callbacks.Callback):
    
    def on_epoch_end(self, epoch, logs=None):
        keys = list(logs.keys())
        with valohai.metadata.logger() as logger:
            logger.log("epoch", epoch)
            for key in keys:
                logger.log(key, logs[key])

In [10]:
model = tf.keras.models.Sequential()

model.add(tf.keras.layers.Dense(10, input_shape=(4,), activation='relu', name='fc1'))
model.add(tf.keras.layers.Dense(10, activation='relu', name='fc2'))
model.add(tf.keras.layers.Dense(3, activation='softmax', name='output'))

# Adam optimizer with learning rate of 0.001
optimizer = tf.keras.optimizers.Adam(lr=learning_rate)
model.compile(optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

print('Neural Network Model Summary: ')
print(model.summary())

Neural Network Model Summary: 
Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 fc1 (Dense)                 (None, 10)                50        
                                                                 
 fc2 (Dense)                 (None, 10)                110       
                                                                 
 output (Dense)              (None, 3)                 33        
                                                                 
Total params: 193
Trainable params: 193
Non-trainable params: 0
_________________________________________________________________
None


2022-01-10 18:07:02.473217: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /home/choprasu/tmp/sqlite3/lib/
2022-01-10 18:07:02.473301: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-01-10 18:07:02.473337: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (a-2xjpj9pnvruee): /proc/driver/nvidia/version does not exist
2022-01-10 18:07:02.473730: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/choprasu/Desktop/ml/env/lib64/python3.7/sit

In [11]:
model.fit(train_x, train_y, verbose=2, batch_size=batch_size, epochs=epochs, callbacks=[CustomCallback()])

Epoch 1/5

{"epoch": 0, "loss": 1.2295314073562622, "accuracy": 0.3083333373069763}
2/2 - 1s - loss: 1.2295 - accuracy: 0.3083 - 523ms/epoch - 261ms/step
Epoch 2/5

{"epoch": 1, "loss": 1.2060948610305786, "accuracy": 0.32499998807907104}
2/2 - 0s - loss: 1.2061 - accuracy: 0.3250 - 5ms/epoch - 2ms/step
Epoch 3/5

{"epoch": 2, "loss": 1.1868776082992554, "accuracy": 0.32499998807907104}
2/2 - 0s - loss: 1.1869 - accuracy: 0.3250 - 6ms/epoch - 3ms/step
Epoch 4/5

{"epoch": 3, "loss": 1.1699867248535156, "accuracy": 0.32499998807907104}
2/2 - 0s - loss: 1.1700 - accuracy: 0.3250 - 5ms/epoch - 3ms/step
Epoch 5/5

{"epoch": 4, "loss": 1.1560242176055908, "accuracy": 0.3333333432674408}
2/2 - 0s - loss: 1.1560 - accuracy: 0.3333 - 10ms/epoch - 5ms/step


In [12]:
# Test on unseen data

results = model.evaluate(test_x, test_y)

print('Final test set loss: {:4f}'.format(results[0]))
print('Final test set accuracy: {:4f}'.format(results[1]))

1/1 [==============================] - 0s 175ms/step - loss: 1.1215 - accuracy: 0.3333
Final test set loss: 1.121497
Final test set accuracy: 0.333333
